this script is an attempt to classify 542 good and bad translations from RusLTC on tf-idf weignted pos n-grams

(just for the hell of it)

self-contained

In [38]:
import sys,os
import pandas as pd
from time import time

import numpy as np
from numpy import dot
from numpy.linalg import norm
from numpy import median
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate, cross_val_score

from sklearn.metrics.pairwise import cosine_distances
from scipy.spatial import distance
import scipy.stats as sc

%matplotlib inline

In [6]:
def preparedata(directory):
    ourdic = []
    print('Collecting data from the files...')
    for subdir in os.listdir(directory):
        files = [f for f in os.listdir(os.path.join(directory, subdir)) if f.endswith('.pos')]
        for f in files:
            rowdic = {'doc': f.strip(), 'group': subdir}
            doc = open(os.path.join(directory, subdir, f))
            text = doc.read().strip() #.replace('\n', ' ')
            doc.close()
            rowdic['text'] = text
            ourdic.append(rowdic)
    ourdic = pd.DataFrame(ourdic)
    return ourdic

In [7]:
def featureselection(x, labels, n_features):
    # Feature selection
    ff = SelectKBest(k=n_features).fit(x, labels)
    newdata = SelectKBest(k=n_features).fit_transform(x, labels)
    top_ranked_features = sorted(enumerate(ff.scores_), key=lambda y: y[1], reverse=True)[:n_features]
    top_ranked_features_indices = [x[0] for x in top_ranked_features]
    return newdata, top_ranked_features_indices

In [8]:
def visual(data, labels, classes):
    # Here goes the 2-D plotting of the data...
    pca = PCA(n_components=2)
    x_r = pca.fit_transform(data)
    plt.figure()
    # consistent colors
    cols = ['red', 'green', 'orange', 'blue', 'grey']
    colors = {}
    for i,name in enumerate(classes):
        colors[name] = cols[i]
#     colors = {'bad': 'red', 'good': 'green'}
    lw = 2

    for target_name in classes:
        plt.scatter(x_r[labels == target_name, 0], x_r[labels == target_name, 1], s=1, color=colors[target_name],
                    label=target_name, alpha=.8, lw=lw)
    plt.legend(loc='best', scatterpoints=1, prop={'size': 15})
    plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    plt.tick_params(axis='y', which='both', left='off', right='off', labelleft='off')
    #plt.savefig('plot.png', dpi=300)
    plt.show()
    plt.close()
    return x_r

In [9]:
datadir = '/home/masha/accuracy/data/pos/'  # The path to where subdirectories with text files are

data_dic = preparedata(datadir)
### this this raw input texts and labels
Xraw3 = data_dic['text']
# print(Xraw[0])
Y3 = data_dic['group']
print(set(Y3))

{'bad', 'good', 'source'}


In [42]:
## lose sources for tf-idf baseline
print(data_dic.shape)
data_dic = data_dic[data_dic['group'] != 'source']
print(data_dic.shape)
Xraw = data_dic['text'].values
print(Xraw[0])
Y = data_dic['group']
print(set(Y))

(542, 3)
(542, 3)
DET NOUN ADP PROPN NOUN VERB NOUN VERB PUNCT SCONJ PRON ADJ NOUN VERB ADP PRON PUNCT SCONJ VERB ADJ NOUN PUNCT
NOUN PART ADJ PUNCT SCONJ PRON PRON VERB ADP ADJ NOUN PUNCT
ADV PRON PART ADJ PUNCT
NOUN PROPN PROPN NUM NOUN NUM PROPN PROPN VERB ADP DET ADJ NOUN CCONJ ADV VERB PUNCT PUNCT NOUN PUNCT PROPN PUNCT PRON DET PUNCT DET NOUN PUNCT PRON VERB ADP NOUN PUNCT PUNCT
PUNCT PRON ADJ VERB NOUN PUNCT ADP PRON PRON ADV VERB PUNCT PUNCT VERB PRON PUNCT
NOUN PUNCT ADJ NOUN ADP ADJ PROPN PROPN PUNCT ADV PART ADJ PUNCT
PRON VERB VERB DET DET NOUN ADP NOUN VERB NOUN PUNCT
ADV VERB ADP PRON VERB PRON PUNCT SCONJ DET NOUN ADJ AUX AUX VERB ADV ADV ADV PUNCT SCONJ NOUN VERB ADJ NOUN PUNCT PUNCT ADV PRON VERB ADP DET PART NOUN PUNCT SCONJ PART NUM NOUN ADV PUNCT PUNCT
VERB NOUN PUNCT PROPN PROPN PUNCT ADJ NOUN PUNCT DET NOUN VERB NOUN PUNCT VERB ADP ADJ NOUN PUNCT PUNCT NUM NOUN AUX VERB ADP NOUN DET NOUN PUNCT
NUM NOUN ADV PRON PART VERB PUNCT SCONJ PUNCT SCONJ PRON VERB PROPN ADP

In [53]:
# optimize the SVM-model, find best hyper-parameters for both the feature extraction and the classifier
def gridsearch_SVM(x, y, refit_score='f1_score', algo=None, grid=False):
    ## norm='l2', use_idf=True, analyzer='word', sublinear_tf=True
    ## number of 3 element permutations for 17 upos pos tags is 17^3 = 4913 (repetitions allowed)
    ## Maybe tf-idf does not make sense for pos ngrams, because none are specific for some texts and not others? 
    
#     tfidf = TfidfVectorizer(analyzer='word', ngram_range=(3, 3), norm='l2', use_idf=False, stop_words=None, smooth_idf=True)
    ## training took an hour! SVM__C: 1, SVM__gamma: 0.01
    cvec = CountVectorizer(analyzer='word', ngram_range=(3, 3), stop_words=None, lowercase=False)
    
    clf = svm.SVC(decision_function_shape='ovo', kernel='rbf',
              random_state=100, verbose=False, probability=True, class_weight="balanced")
    skf = StratifiedKFold(n_splits=10)
    
    pipeline = Pipeline([
    ('cvec', cvec), #('tfidf', tfidf),
    ('SVM', clf),
    ])
    ## 'tfidf__analyzer': ('word','char'), 'SVM__kernel': ['rbf','linear'], , 'tfidf__use_idf': (True, False), 'tfidf__max_features': (100, 500, 1000),
    parameters = {'SVM__gamma': [0.01, 0.1, 1, 10],'SVM__C': [1, 10, 100]}
    
    if algo == 'SVM':
        if grid:
            grid = GridSearchCV(pipeline, parameters, refit=refit_score, cv=skf, scoring='f1_macro',
                                       n_jobs=4, verbose=1)

            print("Performing grid search...")
            print("pipeline:", [name for name, _ in pipeline.steps])
            print("parameters:")
            print(parameters)
            t0 = time()
            grid.fit(x, y)
            print("done in %0.3fs" % (time() - t0))
            print()

            print("Best score: %0.3f" % grid.best_score_)
            print("Best parameters set:")
            best_parameters = grid.best_estimator_.get_params()
            for param_name in sorted(parameters.keys()):
                print("\t%s: %r" % (param_name, best_parameters[param_name]))

            grid.refit

            predicted = grid.predict(x)
            print("accuracy=", np.mean(predicted == y))
            print("macro f-score", f1_score(y, predicted, average='macro'))
            print("Confusion matrix\n", confusion_matrix(y, predicted))

            print('=====')
            print('Unpack the pipeline and look at some distinctive features and produce cross-validated metrics for the classifier on the weighted vectors')

            vectorizer = grid.best_estimator_.named_steps["cvec"] #"tfidf"
            ## Call the fit() function in order to tokenize and learn a vocabulary from one or more documents, 
            ## and learn the vocabulary and inverse document frequency weightings
            # transform the raw training texts into andataset:
            vectorizer.fit(x)
            # summarize
        #     print(vectorizer.vocabulary_)
        #     print(vectorizer.idf_)
            # encode documents
            X_train = vectorizer.transform(x)

            # find maximum value for each of the features over dataset:
            max_value = X_train.max(axis=0).toarray().ravel()
            sorted_by_tfidf = max_value.argsort()
            # get feature names
            feature_names = np.array(vectorizer.get_feature_names())

            ## either very commonly used across documents, or are only used sparingly, and only in very long documents
            print("features with lowest tfidf")
            print(feature_names[sorted_by_tfidf[:20]])

            ## tend to appear very often in some particular documents
            print("features with highest tfidf")
            print(feature_names[sorted_by_tfidf[-20:]])

            ## PCA and SVC does not support sparse input. If used earlier in the script, it throws up 'matrix' object has no attribute 'toarray'
            X_train = X_train.todense()
            print(type(X_train))

            classifier = grid.best_estimator_.named_steps["SVM"]

            print('=====')
            print('Here goes cross-validation. Please wait a bit...')

            scoring = ['precision_macro', 'recall_macro', 'f1_macro']
            cv_scores = cross_validate(classifier, X_train, y, cv=skf, scoring=scoring, n_jobs=2)


            print("Average Precision on 10-fold cross-validation: %0.3f (+/- %0.3f)" % (
                cv_scores['test_precision_macro'].mean(), cv_scores['test_precision_macro'].std() * 2))
            print("Average Recall on 10-fold cross-validation: %0.3f (+/- %0.3f)" % (
                cv_scores['test_recall_macro'].mean(), cv_scores['test_recall_macro'].std() * 2))
            print("Average F1 on 10-fold cross-validation: %0.3f (+/- %0.3f)" % (
                cv_scores['test_f1_macro'].mean(), cv_scores['test_f1_macro'].std() * 2))

            visual(X_train, y, classifier.classes_)
        
        else:
            cvec = CountVectorizer(analyzer='word', ngram_range=(3, 3), stop_words=None, lowercase=False, 
                                   max_features=1000)
            clf = svm.SVC(decision_function_shape='ovo', kernel='rbf', C=1, gamma=0.01,
              random_state=100, verbose=False, probability=True, class_weight="balanced")
            
            cvec.fit(x)
            X_train = cvec.transform(x)
            print(type(X_train))
            print(X_train.shape)
            
            # find maximum value for each of the features over dataset:
            max_value = X_train.max(axis=0).toarray().ravel()
            sorted_by_tfidf = max_value.argsort()
            # get feature names
            feature_names = np.array(cvec.get_feature_names())

            print("features with highest freqs")
            print(feature_names[sorted_by_tfidf[-20:]])
            
            ## scaling does not change the 100% accuracy
            tfidf_trf = TfidfTransformer(norm=None)
            X_train = tfidf_trf.fit_transform(X_train)
            
            ## or l2-normalize the bag-of-woX_trainrds representation
            X_train = preprocessing.normalize(X_train, axis=0)
            print(type(X_train))
            print(X_train.shape)
#             print(X_train[:3])
            
            X_train = X_train.todense()
            
            print('=====')
            print('Here goes cross-validation. Please wait a bit...')
            
            scores_f = cross_val_score(clf, X_train, y, cv=skf, scoring='f1_macro')  # cv = loo f1_macro
            scores_acc = cross_val_score(clf, X_train, y, cv=skf, scoring='accuracy')

            print("F1 over 10folds: ", scores_f.mean())
            print("Accuracy over 10folds: ", scores_acc.mean())

            preds = cross_val_predict(clf, X_train, y, cv=skf)
            print('Cross-validated estimates for data points')
            print(classification_report(y, preds))

            print('===ConfusionMatrix===')
            cnf_matrix = confusion_matrix(y, preds)
            print(cnf_matrix)
    
    elif algo == 'dummy':
        vectorizer = CountVectorizer(analyzer='word', ngram_range=(3, 3), stop_words=None, lowercase=False)
##### this takes forever!
#         vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(3, 3), norm='l1', use_idf=True, max_features=1000, lowercase=True, stop_words=None, smooth_idf=True)
        vectorizer.fit(x)
        X_train = vectorizer.transform(x)
        strategy='stratified'
        print('\n====DummyBaseline (%s)====' % strategy)
        clf = DummyClassifier(strategy=strategy,random_state=42) # 'stratified','uniform', 'most_frequent'
        
        ## this function is simpler than cross_validate, it returns a list of scores on just one specified metric, rather than a dict and tons of other info in that dict
        scores_f = cross_val_score(clf, X_train, y, cv=skf, scoring='f1_macro')  # cv = loo f1_macro
        scores_acc = cross_val_score(clf, X_train, y, cv=skf, scoring='accuracy')

        print("F1 over 10folds: ", scores_f.mean())
        print("Accuracy over 10folds: ", scores_acc.mean())

        preds = cross_val_predict(clf, X_train, y, cv=skf)
        print('Cross-validated estimates for data points')
        print(classification_report(y, preds))

        print('===ConfusionMatrix===')
        cnf_matrix = confusion_matrix(y, preds)
        print(cnf_matrix)

In [54]:
gridsearch_SVM(Xraw, Y, refit_score='f1_score', algo='SVM', grid=False)

<class 'scipy.sparse.csr.csr_matrix'>
(542, 1000)
features with highest freqs
['PUNCT PRON VERB' 'ADJ NOUN NOUN' 'DET NOUN PUNCT' 'ADJ NOUN ADP'
 'NOUN PUNCT CCONJ' 'VERB NOUN PUNCT' 'NOUN PUNCT ADV' 'NOUN PUNCT PRON'
 'VERB ADJ NOUN' 'ADP NOUN NOUN' 'NOUN PUNCT ADP' 'NOUN PUNCT VERB'
 'VERB ADP NOUN' 'ADP NOUN PUNCT' 'NOUN ADJ NOUN' 'ADP ADJ NOUN'
 'NOUN PUNCT NOUN' 'NOUN NOUN PUNCT' 'NOUN ADP NOUN' 'ADJ NOUN PUNCT']
<class 'scipy.sparse.csc.csc_matrix'>
(542, 1000)
=====
Here goes cross-validation. Please wait a bit...
F1 over 10folds:  1.0
Accuracy over 10folds:  1.0
Cross-validated estimates for data points
              precision    recall  f1-score   support

         bad       1.00      1.00      1.00       213
        good       1.00      1.00      1.00       329

   micro avg       1.00      1.00      1.00       542
   macro avg       1.00      1.00      1.00       542
weighted avg       1.00      1.00      1.00       542

===ConfusionMatrix===
[[213   0]
 [  0 329]]


In [35]:
gridsearch_SVM(Xraw, Y, refit_score='f1_score', algo='dummy')


====DummyBaseline (stratified)====
F1 over 10folds:  0.4863718334635806
Accuracy over 10folds:  0.5221231179721745
Cross-validated estimates for data points
              precision    recall  f1-score   support

         bad       0.38      0.33      0.35       213
        good       0.60      0.65      0.62       329

   micro avg       0.52      0.52      0.52       542
   macro avg       0.49      0.49      0.49       542
weighted avg       0.51      0.52      0.52       542

===ConfusionMatrix===
[[ 70 143]
 [116 213]]
